In [ ]:
import cv2
import numpy as np
from sort import Sort  # Import the SORT tracker

class ObjectDetection:
    def __init__(self, weights_path="dnn_model/yolov4.weights", cfg_path="dnn_model/yolov4.cfg"):
        print("Loading Object Detection")
        print("Running opencv dnn with YOLOv4")
        self.nmsThreshold = 0.3
        self.confThreshold = 0.6
        self.image_size = 224  # Adjust as needed

        # Load Network
        net = cv2.dnn.readNet(weights_path, cfg_path)

        # Enable GPU CUDA
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
        self.model = cv2.dnn_DetectionModel(net)

        self.model.setInputParams(size=(self.image_size, self.image_size), scale=1 / 255)

    def detect(self, frame):
        # Detect objects, only retrieve class IDs, scores, and boxes
        class_ids, scores, boxes = self.model.detect(frame, nmsThreshold=self.nmsThreshold, confThreshold=self.confThreshold)
        return scores, boxes

# Initialize Object Detection
od = ObjectDetection()
cap = cv2.VideoCapture("los_angeles.mp4")
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Initialize SORT tracker
tracker = Sort()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.resize(frame, (640, 480))  # Resize as needed
    
    # Detect objects
    scores, boxes = od.detect(frame)
    
    # Prepare detections for SORT
    # Format: [[x1, y1, x2, y2, score], ...]
    detections = []
    for score, box in zip(scores, boxes):
        x, y, w, h = box
        x1, y1, x2, y2 = x, y, x + w, y + h
        detections.append([x1, y1, x2, y2, score])

    # Update SORT tracker
    tracked_objects = tracker.update(np.array(detections))
    
    # Draw tracking results
    for obj in tracked_objects:
        x1, y1, x2, y2, obj_id = obj
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
        cv2.putText(frame, f"ID: {int(obj_id)}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press Esc to exit
        break

cap.release()
cv2.destroyAllWindows()


Loading Object Detection
Running opencv dnn with YOLOv4


[ WARN:0@1.784] global net_impl.cpp:178 setUpNet DNN module was not built with CUDA backend; switching to CPU


ValueError: operands could not be broadcast together with shapes (0,) (1,3) 

: 